In [1]:
import findspark
findspark.init()
import pyspark

from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [2]:
# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext
sqlContext = SQLContext(sc)

### Datos
https://www.kaggle.com/noahgift/social-power-nba

In [3]:
nba = sqlContext.read.csv('./Datasets/nba.csv', header=True).rdd.cache()
twitter = sqlContext.read.csv('./Datasets/twitter.csv', header=True).rdd.cache()

In [4]:
nba.first()

Row(PLAYER='Russell Westbrook', TEAM='OKC', AGE='28', GP='81', W='46', L='35', MIN='34.6', OFFRTG='107.9', DEFRTG='104.6', NETRTG='3.3', AST%='54.3', AST/TO='1.92', AST RATIO='23.4', OREB%='5.3', DREB%='27.9', REB%='16.7', TO RATIO='12.2', EFG%='47.6', TS%='55.4', USG%='40.8', PACE='102.31', PIE='23')

In [5]:
twitter.first()

Row(PLAYER='Russell Westbrook', TWITTER_FAVORITE_COUNT='2130.5', TWITTER_RETWEET_COUNT='559')

In [6]:
nba.count()

486

In [7]:
twitter.count()

329

### Ejercicios

1.   Cuantos jugadores hay por equipo?
2.   Obtener el jugador con mayor edad del equipo Cleveland (CLE).
3.   Obtener promedio de edad de todos los jugadores.
4.   Obtener promedio de edad por equipo ordenados de mayor a menor.
5.   Indicar para cada equipo cuales son los jugadores que poseen mayor edad que el promedio de edad del equipo.
6.   Obtener el top 10 de jugadores en base al puntaje de popularidad calculado como: GP \* 5 + W \* 10 + TWITTER_FAVORITE_COUNT / 2 + TWITTER_RETWEET_COUNT
7.   Calcular el promedio del puntaje de popularidad general teniendo en cuenta a todos los jugadores.
8.   Calcular el promedio del puntaje de popularidad para cada equipo.
9.   Calcular para cada equipo cuantos jugadores superan el promedio de popularidad general (calculado en el punto 7).
10.   Para cada jugador obtener el porcentaje de partidos ganados sobre partidos jugados. Calcular el promedio para cada equipo, teniendo en cuenta solo a los jugadores que hayan jugado al menos el 20% de los juegos del promedio de juegos jugados para los compañeros de su equipo. Obtener los 5 jugadores que poseen los promedios más superiores con relación al promedio de su equipo (si el promedio del equipo es 0,70 y el valor para el jugador es 0,82, el número a tener en cuenta es (0,82-0,70)/0,70).

1. Cuantos jugadores hay por equipo?

In [8]:
team_by_age = nba.map(lambda x: (x.TEAM, 1)).reduceByKey(lambda x, y: x + y).cache()

In [9]:
teams = team_by_age.count()
teams

30

In [10]:
team_by_age.collect()

[('OKC', 15),
 ('DET', 15),
 ('BOS', 15),
 ('NOP', 17),
 ('HOU', 15),
 ('GSW', 16),
 ('CLE', 19),
 ('LAC', 15),
 ('MIL', 17),
 ('SAS', 16),
 ('PHI', 16),
 ('MIN', 15),
 ('DEN', 17),
 ('CHI', 16),
 ('MIA', 14),
 ('UTA', 15),
 ('MEM', 16),
 ('POR', 14),
 ('TOR', 16),
 ('WAS', 17),
 ('ATL', 18),
 ('ORL', 18),
 ('PHX', 17),
 ('CHA', 17),
 ('IND', 16),
 ('DAL', 19),
 ('NYK', 15),
 ('BKN', 18),
 ('SAC', 16),
 ('LAL', 16)]

2. Obtener el jugador con mayor edad del equipo Cleveland (CLE).

In [11]:
nba.filter(lambda x: x.TEAM == 'CLE').reduce(lambda x, y: x if x.AGE > y.AGE else y)

Row(PLAYER='Chris Andersen', TEAM='CLE', AGE='38', GP='12', W='8', L='4', MIN='9.5', OFFRTG='112.7', DEFRTG='108.4', NETRTG='4.3', AST%='6.3', AST/TO='1', AST RATIO='13.1', OREB%='9.8', DREB%='19.1', REB%='15', TO RATIO='13.1', EFG%='40.9', TS%='49.7', USG%='13.6', PACE='96.5', PIE='6.3')

3. Obtener promedio de edad de todos los jugadores.

In [12]:
total = nba.map(lambda x: int(x.AGE)).reduce(lambda x, y: x + y)

In [13]:
count = nba.count()

In [14]:
total/count

26.847736625514404

4. Obtener promedio de edad por equipo ordenados de mayor a menor.

In [15]:
teams_avg_age = nba.map(lambda x: (x.TEAM, (int(x.AGE), 1))).reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))\
    .map(lambda x: (x[0], x[1][0] / x[1][1]))

In [16]:
teams_avg_age.takeOrdered(teams, lambda x: -x[1])

[('CLE', 30.42105263157895),
 ('LAC', 30.066666666666666),
 ('SAS', 29.0),
 ('GSW', 28.625),
 ('ATL', 28.5),
 ('MEM', 27.8125),
 ('IND', 27.5),
 ('MIA', 27.357142857142858),
 ('NYK', 27.266666666666666),
 ('DAL', 27.263157894736842),
 ('NOP', 26.88235294117647),
 ('UTA', 26.733333333333334),
 ('BKN', 26.555555555555557),
 ('LAL', 26.5),
 ('HOU', 26.466666666666665),
 ('SAC', 26.4375),
 ('DEN', 26.352941176470587),
 ('MIL', 26.294117647058822),
 ('OKC', 26.133333333333333),
 ('MIN', 26.133333333333333),
 ('WAS', 26.058823529411764),
 ('CHI', 26.0),
 ('DET', 25.933333333333334),
 ('CHA', 25.823529411764707),
 ('ORL', 25.77777777777778),
 ('BOS', 25.733333333333334),
 ('TOR', 25.6875),
 ('PHX', 25.58823529411765),
 ('PHI', 25.1875),
 ('POR', 24.571428571428573)]

5. Indicar para cada equipo cuales son los jugadores que poseen mayor edad que el promedio de edad del equipo.

In [17]:
nba_by_age = nba.map(lambda x: (x.TEAM, (x.PLAYER, int(x.AGE)))).cache()
nba_by_age.take(1)

[('OKC', ('Russell Westbrook', 28))]

In [18]:
nba_by_age.join(teams_avg_age).filter(lambda x: x[1][0][1] > x[1][1]).groupByKey()\
          .map(lambda x: (x[0], list(x[1]))).collect()


[('DET',
  [(('Boban Marjanovic', 28), 25.933333333333334),
   (('Beno Udrih', 34), 25.933333333333334),
   (('Ish Smith', 28), 25.933333333333334),
   (('Jon Leuer', 28), 25.933333333333334),
   (('Reggie Jackson', 27), 25.933333333333334),
   (('Aron Baynes', 30), 25.933333333333334),
   (('Reggie Bullock', 26), 25.933333333333334),
   (('Marcus Morris', 27), 25.933333333333334)]),
 ('BOS',
  [(('Isaiah Thomas', 28), 25.733333333333334),
   (('Al Horford', 31), 25.733333333333334),
   (('Kelly Olynyk', 26), 25.733333333333334),
   (('Jae Crowder', 26), 25.733333333333334),
   (('Avery Bradley', 26), 25.733333333333334),
   (('Amir Johnson', 30), 25.733333333333334),
   (('Tyler Zeller', 27), 25.733333333333334),
   (('Gerald Green', 31), 25.733333333333334),
   (('Jonas Jerebko', 30), 25.733333333333334)]),
 ('HOU',
  [(('James Harden', 27), 26.466666666666665),
   (('Lou Williams', 30), 26.466666666666665),
   (('Nene', 34), 26.466666666666665),
   (('Patrick Beverley', 28), 26.4666

6. Obtener el top 10 de jugadores en base al puntaje de popularidad calculado como: GP * 5 + W * 10 + TWITTER_FAVORITE_COUNT / 2 + TWITTER_RETWEET_COUNT

In [19]:
nba.take(1)

[Row(PLAYER='Russell Westbrook', TEAM='OKC', AGE='28', GP='81', W='46', L='35', MIN='34.6', OFFRTG='107.9', DEFRTG='104.6', NETRTG='3.3', AST%='54.3', AST/TO='1.92', AST RATIO='23.4', OREB%='5.3', DREB%='27.9', REB%='16.7', TO RATIO='12.2', EFG%='47.6', TS%='55.4', USG%='40.8', PACE='102.31', PIE='23')]

In [20]:
twitter.take(1)

[Row(PLAYER='Russell Westbrook', TWITTER_FAVORITE_COUNT='2130.5', TWITTER_RETWEET_COUNT='559')]

In [21]:
players = nba.map(lambda x: (x.PLAYER, (float(x.GP), float(x.W), x.TEAM))).cache() #Team lo voy a necesitar para 8

In [22]:
twitter_by_player = twitter.filter(lambda x: x.TWITTER_FAVORITE_COUNT is not None and x.TWITTER_RETWEET_COUNT is not None)\
                            .map(lambda x: (x.PLAYER, (float(x.TWITTER_FAVORITE_COUNT), float(x.TWITTER_RETWEET_COUNT))))

In [23]:
twitter_players = players.leftOuterJoin(twitter_by_player)
twitter_players.take(5)

[('Boban Marjanovic', ((35.0, 16.0, 'DET'), None)),
 ('Anthony Davis', ((75.0, 31.0, 'NOP'), (368.0, 104.0))),
 ('James Harden', ((81.0, 54.0, 'HOU'), (969.0, 321.5))),
 ('Kevin Durant', ((62.0, 51.0, 'GSW'), (1425.5, 366.0))),
 ('Chris Paul', ((61.0, 43.0, 'LAC'), (829.0, 178.5)))]

In [24]:
popularity = twitter_players.map(lambda x: (x[1][0][2], x[0], x[1][0][0] * 5 + x[1][0][1] * 10 + (0 if x[1][1] is None else x[1][1][0] / 2 + x[1][1][1])))
popularity.first()

('DET', 'Boban Marjanovic', 335.0)

In [25]:
popularity.takeOrdered(10, lambda x: -x[2])

[('GSW', 'Stephen Curry', 10077.0),
 ('PHI', 'Joel Embiid', 6652.25),
 ('CLE', 'LeBron James', 5148.25),
 ('NYK', 'Derrick Rose', 4016.5),
 ('SAS', 'Kawhi Leonard', 2977.25),
 ('OKC', 'Russell Westbrook', 2489.25),
 ('CLE', 'Kyrie Irving', 2295.5),
 ('GSW', 'Kevin Durant', 1898.75),
 ('DAL', 'Dirk Nowitzki', 1782.5),
 ('HOU', 'James Harden', 1751.0)]

7. Calcular el promedio del puntaje de popularidad general teniendo en cuenta a todos los jugadores.

In [26]:
total_pop = popularity.map(lambda x: x[2]).reduce(lambda x, y: x + y)

In [27]:
count

486

In [28]:
pop_avg = total_pop / count
pop_avg

646.645061728395

8. Calcular el promedio del puntaje de popularidad para cada equipo.

In [29]:
popularity.map(lambda x: (x[0], (x[2], 1))).reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))\
          .map(lambda x: (x[0], x[1][0] / x[1][1])).collect()

[('DET', 551.3166666666667),
 ('HOU', 701.5),
 ('GSW', 1553.21875),
 ('MIL', 586.3235294117648),
 ('SAS', 959.546875),
 ('MIN', 533.8666666666667),
 ('DEN', 530.0735294117648),
 ('CHI', 613.953125),
 ('UTA', 736.3666666666667),
 ('BOS', 732.15),
 ('ATL', 543.8611111111111),
 ('ORL', 458.0),
 ('TOR', 685.21875),
 ('PHX', 434.52941176470586),
 ('CHA', 469.0),
 ('POR', 636.8035714285714),
 ('NYK', 795.0666666666667),
 ('LAL', 529.203125),
 ('NOP', 412.8382352941176),
 ('LAC', 744.45),
 ('PHI', 818.921875),
 ('MIA', 593.4464285714286),
 ('OKC', 771.5),
 ('MEM', 572.6875),
 ('CLE', 889.2894736842105),
 ('DAL', 506.80263157894734),
 ('BKN', 436.5833333333333),
 ('WAS', 602.4558823529412),
 ('IND', 601.046875),
 ('SAC', 512.21875)]

9. Calcular para cada equipo cuantos jugadores superan el promedio de popularidad general (calculado en el punto 7).

In [30]:
popularity.map(lambda x: (x[0], 1 if x[2] > pop_avg else 0)).reduceByKey(lambda x, y: x + y).take(count)

[('DET', 8),
 ('HOU', 10),
 ('GSW', 14),
 ('MIL', 8),
 ('SAS', 12),
 ('MIN', 6),
 ('DEN', 5),
 ('CHI', 8),
 ('UTA', 8),
 ('BOS', 9),
 ('ATL', 9),
 ('ORL', 8),
 ('TOR', 10),
 ('PHX', 2),
 ('CHA', 7),
 ('POR', 10),
 ('NYK', 7),
 ('LAL', 5),
 ('NOP', 5),
 ('LAC', 10),
 ('PHI', 5),
 ('MIA', 8),
 ('OKC', 11),
 ('MEM', 8),
 ('CLE', 8),
 ('DAL', 8),
 ('BKN', 1),
 ('WAS', 9),
 ('IND', 9),
 ('SAC', 5)]

10. Para cada jugador obtener el porcentaje de partidos ganados sobre partidos jugados. Calcular el promedio para cada equipo, teniendo en cuenta solo a los jugadores que hayan jugado al menos el 20% de los juegos del promedio de juegos jugados para los compañeros de su equipo. Obtener los 5 jugadores que poseen los promedios más superiores con relación al promedio de su equipo (si el promedio del equipo es 0,70 y el valor para el jugador es 0,82, el número a tener en cuenta es (0,82-0,70)/0,70).

In [31]:
nba.take(1)

[Row(PLAYER='Russell Westbrook', TEAM='OKC', AGE='28', GP='81', W='46', L='35', MIN='34.6', OFFRTG='107.9', DEFRTG='104.6', NETRTG='3.3', AST%='54.3', AST/TO='1.92', AST RATIO='23.4', OREB%='5.3', DREB%='27.9', REB%='16.7', TO RATIO='12.2', EFG%='47.6', TS%='55.4', USG%='40.8', PACE='102.31', PIE='23')]

In [32]:
nba.map(lambda x: (x.TEAM, (x.PLAYER, int(x.W) / int(x.GP) * 100))).take(5)

[('OKC', ('Russell Westbrook', 56.79012345679012)),
 ('DET', ('Boban Marjanovic', 45.714285714285715)),
 ('BOS', ('Demetrius Jackson', 20.0)),
 ('NOP', ('Anthony Davis', 41.333333333333336)),
 ('HOU', ('James Harden', 66.66666666666666))]